In [1]:
from mmdeploy.apis.utils import build_task_processor
from mmdeploy.utils import get_input_shape, load_config
import torch
from PIL import Image
import numpy as np
import colorsys
from skimage.measure import find_contours
from PIL import Image
import random


deploy_cfg = 'D:/Extensions/openmmlab/mmdeploy/configs/mmdet/instance-seg/instance-seg_rtmdet-ins_onnxruntime_static-640x640.py'
model_cfg = 'D:/Extensions/openmmlab/mmdetection/configs/rtmdet/rtmdet-ins_tiny_8xb32-300e_coco.py'
device = 'cpu'
backend_model = ['./ckpts/end2end.onnx']

In [2]:
# read deploy_cfg and model_cfg
deploy_cfg, model_cfg = load_config(deploy_cfg, model_cfg)

# build task and backend model
task_processor = build_task_processor(model_cfg, deploy_cfg, device)
model = task_processor.build_backend_model(backend_model)

# process input image
input_shape = get_input_shape(deploy_cfg)

03/28 15:41:48 - mmengine - WARNING - Failed to search registry with scope "mmdet" in the "Codebases" registry tree. As a workaround, the current "Codebases" registry in "mmdeploy" is used to build instance. This may cause unexpected failure when running the built modules. Please check whether "mmdet" is a correct scope, or whether the registry is initialized.
03/28 15:41:48 - mmengine - WARNING - Failed to search registry with scope "mmdet" in the "mmdet_tasks" registry tree. As a workaround, the current "mmdet_tasks" registry in "mmdeploy" is used to build instance. This may cause unexpected failure when running the built modules. Please check whether "mmdet" is a correct scope, or whether the registry is initialized.
03/28 15:41:48 - mmengine - WARNING - Failed to search registry with scope "mmdet" in the "backend_detectors" registry tree. As a workaround, the current "backend_detectors" registry in "mmdeploy" is used to build instance. This may cause unexpected failure when running

In [3]:
def random_colors(N, bright=True):
    """
    Generate random colors.
    To get visually distinct colors, generate them in HSV space then
    convert to RGB.
    """
    brightness = 1.0 if bright else 0.7
    hsv = [(i / N, 1, brightness) for i in range(N)]
    colors = list(map(lambda c: colorsys.hsv_to_rgb(*c), hsv))
    random.shuffle(colors)
    return colors

In [21]:
image = "./test/test6.jpg"

image = Image.open(image)
if image.format == "PNG":
    image = image.convert('RGB')
image = np.array(image)

model_inputs, _ = task_processor.create_input(image, input_shape)
with torch.no_grad():
    result = model.test_step(model_inputs)

time:  0.3906521797180176


In [22]:
masks = np.array(result[0].pred_instances.masks).astype('int32')
labels = np.array(result[0].pred_instances.labels).astype('int32')
bboxes = np.array(result[0].pred_instances.bboxes).astype('int32')
scores = np.array(result[0].pred_instances.scores).astype('float32')
h, w = result[0].ori_shape

In [26]:
colors = random_colors(masks.shape[0])
centers = []
res_meta = dict()
padding = 2
idx = 0
for i in range(masks.shape[0]):
    
    if scores[i] < 0.3 or labels[i] not in [2, 7]:
        break

    left, top, right, bottom = bboxes[i]
    colorized_mask = np.zeros((h+2*padding, w+2*padding, 4))
    
    # ignore overlapping bboxes
    flag = False
    _c = [(right+left)//2, (bottom+top)//2]
    for c in centers:
        if abs(_c[0] - c[0]) <= 5 and abs(_c[1] - c[1]) <= 5:
            flag = True
            break
    if flag:
        continue
    centers.append(_c)
    
    # Colorize Inst Mask
    alpha = 0.3
    color = np.array(list(colors[i]) + [alpha]).reshape((1, 1, 4))
    colorized_mask[padding:padding+h, padding:padding+w, :4] = np.where(
        np.expand_dims(masks[i] == 1, -1),
        color * 255,
        colorized_mask[padding:padding+h, padding:padding+w, :4])

    # Draw Mask Polygon
    contours = find_contours(masks[i], 0.5)[0].astype('int32')

    colorized_mask[contours[:, 0], contours[:, 1], :] = np.array([255, 255, 255, 255])
    colorized_mask[contours[:, 0], contours[:, 1]+1, :] = np.array([255, 255, 255, 255])
    colorized_mask[contours[:, 0], contours[:, 1]-1, :] = np.array([255, 255, 255, 255])
    colorized_mask[contours[:, 0]+1, contours[:, 1], :] = np.array([255, 255, 255, 255])
    colorized_mask[contours[:, 0]-1, contours[:, 1], :] = np.array([255, 255, 255, 255])
    
    # Clip and Save
    my1, my2, mx1, mx2 = max(0, top-5),min(h, bottom+5),max(0, left-5),min(w, right+5)
    m_path = "./output/masks-single/m{}.png".format(idx)
    im_path = "./output/images-single/im{}.png".format(idx)
    res_meta[str(idx)] = dict(
        x1=mx1, x2=mx2, y1=my1, y2=my2,
        m_path=m_path,
        im_path=im_path
    )

    Image.fromarray(
        colorized_mask[padding:h+padding, padding:w+padding, :][my1:my2, mx1:mx2, :].astype('uint8'), 
        mode='RGBA'
    ).save(m_path)

    Image.fromarray(
        np.concatenate([
            image[my1:my2, mx1:mx2, :].astype('uint8'),
            np.expand_dims(masks[i][my1:my2, mx1:mx2] * 255, axis=-1).astype('uint8')
        ], axis=-1), 
        mode='RGBA'
    ).save(im_path)

    idx += 1

print(res_meta)

{'0': {'x1': 278, 'x2': 496, 'y1': 229, 'y2': 358, 'm_path': './output/masks-single/m0.png', 'im_path': './output/images-single/im0.png'}, '1': {'x1': 52, 'x2': 163, 'y1': 157, 'y2': 216, 'm_path': './output/masks-single/m1.png', 'im_path': './output/images-single/im1.png'}, '2': {'x1': 124, 'x2': 332, 'y1': 210, 'y2': 302, 'm_path': './output/masks-single/m2.png', 'im_path': './output/images-single/im2.png'}, '3': {'x1': 502, 'x2': 586, 'y1': 163, 'y2': 219, 'm_path': './output/masks-single/m3.png', 'im_path': './output/images-single/im3.png'}, '4': {'x1': 361, 'x2': 437, 'y1': 153, 'y2': 207, 'm_path': './output/masks-single/m4.png', 'im_path': './output/images-single/im4.png'}, '5': {'x1': 147, 'x2': 232, 'y1': 156, 'y2': 212, 'm_path': './output/masks-single/m5.png', 'im_path': './output/images-single/im5.png'}, '6': {'x1': 283, 'x2': 352, 'y1': 109, 'y2': 152, 'm_path': './output/masks-single/m6.png', 'im_path': './output/images-single/im6.png'}, '7': {'x1': 220, 'x2': 286, 'y1': 